In [1]:
import pandas as pd
import numpy as np

#Change path to
TAQ_MS_FILE = "/home/asyed/Documents/Spring 2024/MS&E 349/Volume Prediction/TAQ_Millisecond_AAPL_2023.csv"
chunksize = 2**16  # Size of chunks to read at a time

#Converters to ensure data types are correctly handled
converters = {
    'SIZE': np.int64,
    'PRICE': np.float64,
    'TR_CORR': np.int64,
    'TR_SEQNUM': np.int64,
    'TR_ID': np.int64,
}

In [2]:
chunks = []  #List to hold processed data from each chunk

#Read and process the full-sized data file
for chunk in pd.read_csv(TAQ_MS_FILE, chunksize=chunksize, converters=converters):
    #Create a 'datetime' column by combining 'DATE' and 'TIME_M'
    chunk['datetime'] = pd.to_datetime(chunk['DATE'] + ' ' + chunk['TIME_M'])
    chunk.set_index('datetime', inplace=True)  #'datetime' is now our index
    chunk['TURNOVER'] = chunk['SIZE'] * chunk['PRICE'] #Turnover for each transaction
    #Aggregate data by hourly intervals
    grouped = chunk.groupby(pd.Grouper(freq='1h')).agg(
        TURNOVER=pd.NamedAgg(column="TURNOVER", aggfunc="sum")  #Aggregate turnover
    )
    chunks.append(grouped)  


hourly_data = pd.concat(chunks)
#Remove duplicates by summing turnovers for each hour again 
hourly_data = hourly_data.groupby(hourly_data.index).sum()

In [3]:
#Rolling median of the trailing 30 days of turnovers
hourly_data['date'] = hourly_data.index.date  # Extract date from datetime index
daily_turnover = hourly_data.groupby('date')['TURNOVER'].sum()  # Sum turnovers by day
rolling_median_30d = daily_turnover.rolling(window=30).median()  # Rolling median

In [4]:
#This fn normalizes hourly turnovers by the rolling median of the preceding 30 trading days
def normalize_turnovers(df, rolling_median):
    normalized_turnovers = []
    for date, group in df.groupby(df.index.date):
        if date in rolling_median.index:
            median_value = rolling_median.loc[date]
            normalized_turnovers.extend(group['TURNOVER'] / median_value)
        else:
            normalized_turnovers.extend([np.nan] * len(group))
    return pd.Series(normalized_turnovers, index=df.index)

normalized_turnovers = normalize_turnovers(hourly_data, rolling_median_30d)
hourly_data['Normalized_TURNOVER'] = normalized_turnovers

In [5]:
hourly_data = hourly_data.dropna(subset=['Normalized_TURNOVER'])
hourly_data.drop(columns=['date'], inplace=True)

In [6]:
#Just a sanity check. Let's find the first row where TURNOVER isn't zero
first_non_zero_turnover = hourly_data[hourly_data['TURNOVER'] != 0].iloc[0]
first_non_zero_date = first_non_zero_turnover.name.date()

#Find and print all turnovers for the first non-zero turnover date (so all the trading hours for that day)
non_zero_turnovers_on_date = hourly_data[(hourly_data.index.date == first_non_zero_date) & (hourly_data['TURNOVER'] != 0)]

print("First non-zero turnover entry:")
print(first_non_zero_turnover)

print(f"All non-zero turnovers for {first_non_zero_date}:")
print(non_zero_turnovers_on_date)

#Helps to get the resulting data to as a new CSV file
hourly_data.to_csv("/home/asyed/Documents/Spring 2024/MS&E 349/Volume Prediction/TAQ_Hourly_AAPL_2023_normalized.csv")

print(f"Normalized hourly turnover data saved to /home/asyed/Documents/Spring 2024/MS&E 349/Volume Prediction/TAQ_Hourly_AAPL_2023_normalized.csv")

First non-zero turnover entry:
TURNOVER               1.103893e+09
Normalized_TURNOVER    1.455840e-01
Name: 2023-02-01 09:00:00, dtype: float64
All non-zero turnovers for 2023-02-01:
                         TURNOVER  Normalized_TURNOVER
datetime                                              
2023-02-01 09:00:00  1.103893e+09             0.145584
2023-02-01 10:00:00  1.275860e+09             0.168263
2023-02-01 11:00:00  7.645183e+08             0.100826
2023-02-01 12:00:00  8.913491e+08             0.117553
2023-02-01 13:00:00  7.059671e+08             0.093105
2023-02-01 14:00:00  2.150677e+09             0.283636
2023-02-01 15:00:00  2.710917e+09             0.357522
Normalized hourly turnover data saved to /home/asyed/Documents/Spring 2024/MS&E 349/Volume Prediction/TAQ_Hourly_AAPL_2023_normalized.csv
